# Advent of code 2020

This notebook contains my (somewhat documented) solutions to advent of code 2020. For
each day, I've tried to summarize the solutions into short explanation, understandable
code blocks and, if needed, have added some extra information in code docstrings.

If you want to tinker around with the solutions, you can install needed libraries
with either [poetry](https://python-poetry.org/) or just plain `pip install .`. Some
extra setup is also needed for the awesome [advent-of-code-data](
https://pypi.org/project/advent-of-code-data/) library, which I'm using for getting input. See
project docs for information.

## Setup and shared functions

This first block is used to import all libraries, define all special function and create all types needed
in the notebook. These are mainly for parsing data into different forms.

In [54]:

import re
from aocd import get_data
from collections import Counter, defaultdict
from itertools import combinations
from typing import Any, Iterable, Callable, Optional, Literal, Iterator
from operator import mul
from functools import lru_cache, reduce
from dataclasses import dataclass

def data_to_nums(data: str) -> list[int]:
    """Parse integers from given aocd data."""
    return [int(x) for x in re.findall(r"(\d+)", data)]

def data_to_lines(data: str) -> list[str]:
    """Parse lines from given aocd data."""
    return data.splitlines()

def data_to_blocks(data: str) -> list[str]:
    """Parse blocks that are separated by empty new lines from given aocd data."""
    return [block for block in data.split("\n\n")]

def get_first(iterable: Iterable, condition: Callable) -> Any:
    """Get first object from iterable that matches given condition."""
    return next(thing for thing in iterable if condition(thing))

def multiply(numbers_to_multiply: Iterable[int]) -> int:
    """Multiply iterable of integers together."""
    return reduce(mul, numbers_to_multiply, 1)

Vector = tuple[int, int]
Mode = Literal["first", "second"]

## Day 1

**Task**: find the entries that sum to 2020.

Input size is quite small, just hundred numbers, so brute-forcing through every possible
combination is fast enough. Solve-function gets first k-tuple where sum of tuple
matches given goal.

In [55]:
numbers = data_to_nums(get_data(day=1, year=2020))

def solve(goal: int, number_of_entries: int) -> int:
    return multiply(get_first(combinations(numbers, number_of_entries),
                              lambda *combination: sum(*combination) == goal))

print("Part 1:", solve(goal=2020, number_of_entries=2))
print("Part 2:", solve(goal=2020, number_of_entries=3))

Part 1: 1010299
Part 2: 42140160


## Day 2

**Task**: find amount of passwords that match their counterpart rule.

Ah, classic aocd-style input format! Luckily, regex is here to help. The first part is
handled with Counter from collections library, the second part with exclusive or (XOR)
since either, but not both positions must equal to char.

In [3]:
lines = data_to_lines(get_data(day=2, year=2020))
first, second = 0, 0

for line in lines:
    low, high, char, string = re.match(r"(\d+)-(\d+) ([a-z]): ([a-z]+)", line).groups()
    low, high = int(low), int(high)
    if low <= Counter(string)[char] <= high:
        first += 1
    if (string[low - 1] == char) ^ (string[high - 1] == char):
        second += 1

print("Part 1:", first)
print("Part 2:", second)

Part 1: 524
Part 2: 485


## Day 3

**Task**: count the number of trees encountered while traversing the map.

Important part here is the "jump around" the x-axis, which is done by counting x
with modulo width.

In [4]:
grid = data_to_lines(get_data(day=3, year=2020))

def travel(slope: Vector) -> int:
    x, y, counter = 0, 0, 0
    height, width = len(grid), len(grid[0])
    right, down = slope

    while (y := y + down) < height:
        x = (x + right) % width
        if grid[y][x] == "#":
            counter += 1

    return counter

travels = [travel(slope=slope) for slope in [(3,1), (1, 1), (5, 1), (7, 1), (1, 2)]]

print("Part 1:", travels[0])
print("Part 2:", multiply(travels))

Part 1: 280
Part 2: 4355551200


## Day 4

**Task**: count the number of valid passwords.

Again, regex saves the day! Each password block is ran against seven validators, since
"cid" field is not important. In the first part, it's enough to check that all validator
fields are present. In the second part, has_valid_fields-function checks that all fields
are also valid against the given rules.

In [5]:
Validator = Callable[[str], bool]

passports = data_to_blocks(get_data(day=4, year=2020))
validators: dict[str, Validator] = {
    "byr": lambda v: 1920 <= int(v) <= 2002,
    "iyr": lambda v: 2010 <= int(v) <= 2020,
    "eyr": lambda v: 2020 <= int(v) <= 2030,
    "hgt": lambda v: (
        "cm" in v
        and 150 <= int(v[:-2]) <= 193
        or "in" in v
        and 59 <= int(v[:-2]) <= 76
    ),
    "hcl": lambda v: bool(re.fullmatch(r"^#[0-9a-f]{6}$", v)),
    "ecl": lambda v: v in ["amb", "blu", "brn", "gry", "grn", "hzl", "oth"],
    "pid": lambda v: bool(re.fullmatch(r"^[0-9]{9}$", v)),
}

def has_valid_fields(fields: list[tuple[str, str]]) -> bool:
    return sum(
        validators[field_id](value) for field_id, value in fields if field_id != "cid"
    ) == len(validators)

first, second = 0, 0
for passport in passports:
        fields = re.findall(r"(\w+):(\S+)", passport)
        field_ids = {field_id for field_id, _ in fields}
        if all(field_id in field_ids for field_id in validators):
            first += 1
            if has_valid_fields(fields=fields):
                second += 1
                
print("Part 1:", first)
print("Part 2:", second)

Part 1: 192
Part 2: 101


## Day 5

**Task**: Find out the highest seat ID and your own seat ID.

There's a nice catch in the puzzle: char pairs F/L and B/R aren't really different, since
each can be substituted with 0 or 1. The result is a binary number string which can
be casted to int.

In the second part, helper function finds the first seat id that matches given
conditions.

In [22]:
lines = data_to_lines(get_data(day=5, year=2020))

table = str.maketrans("FBLR", "0101")
seat_ids = {int(line.translate(table), base=2) for line in lines}
ans = max(seat_ids)
print("Part 1:", ans)

print("Part 2:", get_first(range(1, ans), lambda seat: seat not in seat_ids 
                           and seat - 1 in seat_ids and seat + 1 in seat_ids))

Part 1: 858
Part 2: 557


## Day 6

**Task**: Count the number of questions to which anyone/everyone answered "yes".

In the first part, the question is basically "how many unique alphabets there are
between newlines?". In the second part, question is "which alphabets are present on
all lines between newlines?", which can be handled with [set
intersections](https://en.wikipedia.org/wiki/Intersection_(set_theory)).

In [21]:
groups = data_to_blocks(get_data(day=6, year=2020))
first, second = 0, 0

for answers in groups:
    answer_ids = {answer for answer in answers if answer.isalpha()}
    first += len(answer_ids)
    for person in answers.split("\n"):
        answer_ids &= set(person)
    second += len(answer_ids)

print("Part 1:", first)
print("Part 2:", second)

Part 1: 6532
Part 2: 3427


## Day 7

**Task**: Which bags can contain "shiny gold" bag / how many bags does "shiny gold" bag contain?

A bit trickier one! One helpful obvervation is that the rules define a graph, a [directec acyclic graph](https://en.wikipedia.org/wiki/Directed_acyclic_graph) to be precise. This means that both parts can be solved with [depth first search](https://en.wikipedia.org/wiki/Depth-first_search) (dfs).

First dfs uses a graph where edges go from content to container: e.g. "light red bags contain 1 bright white bag, 2 muted yellow bags." results edges `bright white -> light red` and `muted yellow -> light red`. This way it's easy to walk graph through from "shiny gold" node.

Second dfs uses a graph where edges follow the rules: there's and weighted edge from container to content. Graph is walked through from "shiny gold" node and amounts are returned.

In [56]:
Node = str
Edge = tuple[Node, int]
Graph = dict[Node, list[Edge]]
RevGraph = dict[Node, list[Node]]

rules = data_to_lines(get_data(day=7, year=2020))


def create_graphs() -> tuple[Graph, RevGraph]:
    graph: Graph = defaultdict(list)
    reversed_graph: RevGraph = defaultdict(list)
    for rule in rules:
        container, content = rule.split(" bags contain ")
        content = re.findall(r"(\d+) ([\w\s]+(?= ))", content)
        for amount, bag in content:
            graph[container].append((bag, int(amount)))
            reversed_graph[bag].append(container)
    return graph, reversed_graph


def first(graph: Graph) -> int:
    seen: set[Node] = set()
    def dfs(bag: Node) -> None:
        if bag in seen:
            return
        seen.add(bag)
        for content in graph[bag]:
            dfs(content)
    
    dfs("shiny gold")
    return len(seen) - 1


def second(graph: Graph) -> int:
    def dfs(bag: Node, amount: int) -> int:
        if bag not in graph:
            return amount
        new = 0 if bag == "shiny gold" else amount
        for content, num in graph[bag]:
            new += dfs(content, num * amount)
        return new

    return dfs("shiny gold", 1)


graph, reversed_graph = create_graphs()
print("Part 1:", first(graph=reversed_graph))
print("Part 2:", second(graph=graph))

Part 1: 372
Part 2: 8015


## Day 8

**Task**: Find out where given semi-assemby program loops / change one instruction to terminate succesfully.

This reminded me of [Intcode computer](https://adventofcode.com/2019/day/2) tasks from AoC 2019, so I used my last year's Intecode-comp style in this solution too. 

In the first part, loop breaks once some instruction is visited for the second time. In the second part, generator function is used to brute-force all possible tapes where either "jmp" or "nop" is replaced. Function get_first returns value from the first tape that goes beyond its index.

In [57]:
Instruction = tuple[str, int]
Tape = list[Instruction]

instructions = re.findall(r"(\w+) ([\+-]\d+)", get_data(day=8, year=2020))
tape = [(op, int(arg)) for op, arg in instructions]


@dataclass
class Comp:
    tape: Tape
    accumulator: int = 0
    head: int = 0

    def run(self, mode: Mode = "first") -> Optional[int]:
        seen: set[int] = set()
        while self.head not in seen:
            seen.add(self.head)
            try:
                op, arg = self.tape[self.head]
            except IndexError:
                return self.accumulator
            if op == "acc":
                self.accumulator += arg
                self.head += 1
            elif op == "jmp":
                self.head += arg
            elif op == "nop":
                self.head += 1
        if mode == "first":
            return self.accumulator
        

def potential_tapes(tape: Tape) -> Iterator[Tape]:
    replace = {"jmp": "nop", "nop": "jmp"}
    for i, (op, arg) in enumerate(tape):
        if op in replace:
            yield [*tape[:i], (replace[op], arg), *tape[i + 1:]]

print("Part 1:", Comp(tape).run())
            
print("Part 2:", get_first((Comp(potential_tape).run(mode="second") 
                            for potential_tape in potential_tapes(tape)),
                           lambda result: result is not None))

Part 1: 2034
Part 2: 672


## Day 20

We're given `n` tiles, and the problem is to match tiles together to form a sqrt(n)
times sqrt(n) square.

First part can be solved with just matching suitable borders together. Second part is
 a bit trickier, since we're asked to construct the whole square based on matched
 borders and then check the

My idea here is that first borders can be just matched. Then, I'm creating an exact
cover matrix, which solved by numpy-exact-cover -library. After that creating the
solution is quite easy.


In [34]:
Picture = list[str]
TileID = int

def parse_tiles_from_data() -> dict[TileID, Picture]:
    def get_tile_id(header: str) -> TileID:
        return int(re.findall(r"(\d+)", header)[0])

    data = get_data(day=20, year=2020)
    cameras = [camera.splitlines() for camera in data.split("\n\n")]
    return { get_tile_id(header): pic for (header, *pic) in cameras}

tiles = parse_tiles_from_data()